In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cleaning bengal or english
!pip install git+https://github.com/csebuetnlp/normalizer

# kenlm for ctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip

!pip install -r /content/drive/MyDrive/audio/whisper/requirements.txt
# 이거 안하면 저장할 때 오류남~
!pip uninstall peft -y
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-56624m0n
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-56624m0n
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6857 sha256=83e52dba62957c3ca01d0488ca83855d43255c5db85f0fc855be9293c0229a66
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2r0pg2i/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186457 sha256=e30e4c612efd9f14c9e580cdc6db2ec5d80d0d

# import

In [3]:
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as tat

import os
import gc
import glob
import sys
from pathlib import Path
import warnings

import typing as tp
from functools import partial
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa
import gc
import jiwer
import pyctcdecode
import kenlm
from bnunicodenormalizer import Normalizer
import evaluate

from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, pipeline
import datasets
from datasets import load_dataset, load_metric, Audio, Dataset
from huggingface_hub import notebook_login
notebook_login()

warnings.filterwarnings('ignore')
torchaudio.set_audio_backend("soundfile")
sys.path.append("/content/drive/MyDrive/audio_classification/whisper/")

# load data

In [95]:
SR = 16000
MAX_LENGTH = 3000
top_dropout = 0.3

In [5]:
all_train_data = load_dataset("audiofolder",
                        name = "audio",
                        data_dir = "/content/drive/MyDrive/audio_classification/dataset/train",
                        split="train[0%:10%]")
all_train_data = all_train_data.train_test_split(test_size=0.2)

Resolving data files:   0%|          | 0/5850 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [96]:
all_train_data = all_train_data.cast_column("audio", Audio(sampling_rate=SR))
all_train_data["train"][0]

{'audio': {'path': '/content/drive/MyDrive/audio_classification/dataset/train/africa/common_voice_en_18429172.wav',
  'array': array([ 2.91038305e-11, -7.27595761e-11, -1.81898940e-11, ...,
         -7.75897206e-06, -1.35245800e-05, -1.00715424e-05]),
  'sampling_rate': 16000},
 'label': 0}

In [97]:
labels = all_train_data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

print(f"all_label_num : {len(label2id)}\n\n {labels}")

# load model

In [98]:
import whisper_cl_model
model, feature_extractor = whisper_cl_model.load_model(label2id, id2label)

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-base-ft-common-language-id and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([45]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([45, 256]) in the checkpoint and torch.Size([4, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# train

In [82]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    print(batch)
    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=SR).input_features[0]

    # encode target text to label ids
    batch["labels"] = batch["label"]
    return batch

encoded_ds = all_train_data.map(prepare_dataset)

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [103]:
encoded_ds = all_train_data.map(prepare_dataset, remove_columns="audio", batched=True)

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [28]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [51]:
training_args = TrainingArguments(
    output_dir = "/content/drive/MyDrive/audio_classification/models/output_1",
    lr_scheduler_type='cosine',
    weight_decay=0.01,
    evaluation_strategy="steps",
    # save_strategy="steps",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    warmup_steps = 50,
    warmup_ratio=0.1,
    # save_steps=10,
    eval_steps=10,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

In [104]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy
10,1.499200,1.562133,0.059829
20,1.459700,1.456606,0.059829
30,1.333700,1.277256,0.085470


TrainOutput(global_step=30, training_loss=1.4552897135416667, metrics={'train_runtime': 132.3414, 'train_samples_per_second': 3.536, 'train_steps_per_second': 0.227, 'total_flos': 1.344803855616e+16, 'train_loss': 1.4552897135416667, 'epoch': 1.0})

# simple test space

In [4]:
import whisper_cl_dataset
import whisper_cl_model
import whisper_cl_train

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

loading dataset...


Resolving data files:   0%|          | 0/5850 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

all_label_num : 4

 ['africa', 'australia', 'canada', 'england']


config.json:   0%|          | 0.00/6.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/615M [00:00<?, ?B/s]

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-medium-fleurs-lang-id and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([102]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([102, 256]) in the checkpoint and torch.Size([4, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [25]:
labels = whisper_cl_model.get_label_from_ds(whisper_cl_train.train_ds)
model, _ = whisper_cl_model.load_model(labels)

all_label_num : 4

 ['africa', 'australia', 'canada', 'england']


Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-base-ft-common-language-id and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([45]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([45, 256]) in the checkpoint and torch.Size([4, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model

WhisperForAudioClassification(
  (encoder): WhisperEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 512)
    (layers): ModuleList(
      (0-5): 6 x WhisperEncoderLayer(
        (self_attn): WhisperAttention(
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affi

In [9]:
import imp
imp.reload(whisper_cl_model)
imp.reload(whisper_cl_dataset)
imp.reload(whisper_cl_train)

loading dataset...


Resolving data files:   0%|          | 0/5850 [00:00<?, ?it/s]

all_label_num : 4

 ['africa', 'australia', 'canada', 'england']


Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-base-ft-common-language-id and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([45]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([45, 256]) in the checkpoint and torch.Size([4, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<module 'whisper_cl_train' from '/content/drive/MyDrive/audio_classification/whisper/whisper_cl_train.py'>

In [5]:
ds = whisper_cl_dataset.main()

loadin dataset from directory...


Resolving data files:   0%|          | 0/5850 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

preprocessing dataset...


Map:   0%|          | 0/936 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'label', 'input_features', 'labels'],
        num_rows: 936
    })
    test: Dataset({
        features: ['audio', 'label', 'input_features', 'labels'],
        num_rows: 234
    })
})

In [11]:
whisper_cl_model.get_label_from_ds(ds)

['africa', 'australia', 'canada', 'england']

In [ ]:
whisper_cl_train.train()

training... !!!!!!


Step,Training Loss,Validation Loss,Accuracy
20,1.398200,0.474941,0.859829


WhisperForAudioClassification(
  (encoder): WhisperEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 512)
    (layers): ModuleList(
      (0-5): 6 x WhisperEncoderLayer(
        (self_attn): WhisperAttention(
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affi